# Sampler modeller nasıl çalışır?

In [ ]:
#----------------------------------------------------------------------------------------#
# Add diffusers path to sys path
import sys
import os
from dotenv import load_dotenv;
# Load the .env file
load_dotenv()
# Get Diffusers path from environment variable
diffusers_path = os.getenv('DIFFUSERS_PATH')
print(f"Diffusers path: {diffusers_path}")
if diffusers_path is None:
    raise ValueError("Please set DIFFUSERS_PATH environment variable to Diffusers path")
sys.path.append(diffusers_path+"/src")
#----------------------------------------------------------------------------------------#

In [ ]:
from diffusers import DDPMPipeline

ddpm = DDPMPipeline.from_pretrained("google/ddpm-celebahq-256").to("cuda")
image = ddpm(num_inference_steps=30).images[0]

display(image)

DDPMPipeline'ı diffusers kullanarak oluşturduk. Peki bu pipeline nasıl çalışır? DDPM Pipeline'ı oluştururken

In [ ]:
from diffusers import DDPMScheduler, UNet2DModel

scheduler = DDPMScheduler.from_pretrained("google/ddpm-cat-256")
model = UNet2DModel.from_pretrained("google/ddpm-cat-256").to("cuda")

scheduler.set_timesteps(30)
print(scheduler.config)

Scheduler zaman adımlarını ayarlamak, bu örnekte 50 eşit aralıklı öğeler içeren bir tensör oluşturur. Her öğe, modelin bir görüntüyü gürültüden arındırdığı bir zaman adımına karşılık gelir. Daha sonra gürültü giderme döngüsünü oluşturduğumuzda, bir görüntüyü gürültüden arındırmak için bu tensör üzerinde yineleme yapacağız.

İstenen çıktıyla aynı şekle sahip rastgele bir gürültü oluşturuyoruz.

In [ ]:
import torch
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

sample_size = 256
noise = torch.randn((1, 3, sample_size, sample_size)).to("cuda")
print(noise.shape)

In [ ]:
input = noise

noisy_sample = []
prev_noisy_sample = []

for time in scheduler.timesteps:
    with torch.no_grad():
        noisy_residual = model(input, time).sample
    noisy_sample.append(noisy_residual)
    previous_noisy_sample = scheduler.step(noisy_residual, time, input).prev_sample
    prev_noisy_sample.append(previous_noisy_sample)
    input = previous_noisy_sample


image = (input / 2 + 0.5).clamp(0, 1)
image = image.cpu().permute(0, 2, 3, 1).numpy()[0]
image = Image.fromarray((image * 255).round().astype("uint8"))
display(image)

num_rows = 3
num_cols = 10
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 6))

# Her bir tensörü ayrı ayrı görüntüleyin
for i, tensor in enumerate(noisy_sample):
    row = i // num_cols
    col = i % num_cols
    tensor = (tensor / 2 + 0.5).clamp(0, 1)
    tensor = tensor.cpu().permute(1,0, 2, 3).numpy()[0]
    axs[row, col].imshow(np.transpose(tensor, (1, 2, 0)))
    axs[row, col].axis('off')

plt.show()
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 6))

for i, tensor in enumerate(prev_noisy_sample):
    row = i // num_cols
    col = i % num_cols
    tensor = (tensor / 2 + 0.5).clamp(0, 1)
    tensor = tensor.cpu().permute(1,0, 2, 3).numpy()[0]
    axs[row, col].imshow(np.transpose(tensor, (1, 2, 0)))
    axs[row, col].axis('off')

plt.show()